## EDA

In [1]:
#importing the libraries
import numpy as np
import pandas as pd
from scipy import stats
import datetime 
import plotly.express as px
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

In [5]:
#Adding scripts path
import sys, os
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from data_preprocessing import data_preprocessing_script
from cleaner import DataCleaner

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\hp\\SalesPrediction\\notebooks\\logs\\data_preprocessing.log'